In [1]:
import optuna
import warnings
import gc
import os
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime
from glob import glob
from tqdm import tqdm

warnings.filterwarnings(action = 'ignore')

In [2]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob("Data/train/KNOW*.csv"))):
    if i == 0:
        train_2017 = pd.read_csv(path)
        pass
    elif i==1:
        train_2018 = pd.read_csv(path)
        pass
    elif i==2:
        train_2019 = pd.read_csv(path)
        pass
    else:
        train_2020 = pd.read_csv(path)
        pass
    
test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob("Data/test/KNOW*.csv"))):
    if i == 0:
        test_2017 = pd.read_csv(path)
        pass
    elif i==1:
        test_2018 = pd.read_csv(path)
        pass
    elif i==2:
        test_2019 = pd.read_csv(path)
        pass
    else:
        test_2020 = pd.read_csv(path)
        pass

### 2017년 데이터 결측치 처리

- 중앙값으로 결측치 처리
- data leakage 고려하여 train 데이터로 결측치 처리

In [3]:
# aq 문항 결측치 처리
for col in train_2017:
        train_2017[col].replace(' ', np.nan, inplace = True)
        if col != 'knowcode':
            test_2017[col].replace(' ', np.nan, inplace = True)

columns = train_2017.columns[1:-1][:82]

for index in range(0,82,2):
    col = columns[index]
    
    before = col
    after = col[:-1] + str((int(col[-1])+1))
    
    train_2017.loc[(train_2017[after].isnull()) & (train_2017[before].notnull()) & (train_2017[before] == (sorted(train_2017[before].unique()))[0]), after] = 0
    test_2017.loc[(test_2017[after].isnull()) & (test_2017[before].notnull()) & (test_2017[before] == (sorted(test_2017[before].unique()))[0]), after] = 0

temp_2017 = train_2017[(train_2017['aq11_2'].notnull()) & (train_2017['aq11_1'] == 3)]
train_2017.loc[train_2017['aq11_2'].isnull(), 'aq11_2'] = temp_2017['aq11_2'].astype('int').quantile(q = 0.5)
test_2017.loc[test_2017['aq11_2'].isnull(), 'aq11_2'] = temp_2017['aq11_2'].astype('int').quantile(q = 0.5)

temp_2017 = train_2017[(train_2017['aq14_2'].notnull()) & (train_2017['aq14_1'] == 3)]
train_2017.loc[(train_2017['aq14_2'].isnull()) & (train_2017['aq14_1'] == 3), 'aq14_2'] = temp_2017['aq14_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq14_2'].isnull()) & (test_2017['aq14_1'] == 3), 'aq14_2'] = temp_2017['aq14_2'].astype('int').quantile(q = 0.5)

temp_2017 = train_2017[(train_2017['aq14_2'].notnull()) & (train_2017['aq14_1'] == 4)]
train_2017.loc[(train_2017['aq14_2'].isnull()) & (train_2017['aq14_1'] == 4), 'aq14_2'] = temp_2017['aq14_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq14_2'].isnull()) & (test_2017['aq14_1'] == 4), 'aq14_2'] = temp_2017['aq14_2'].astype('int').quantile(q = 0.5)

temp_2017 = train_2017[(train_2017['aq27_2'].notnull()) & (train_2017['aq27_1'] == 3)]
train_2017.loc[(train_2017['aq27_2'].isnull()) & (train_2017['aq27_1'] == 3), 'aq27_2'] = temp_2017['aq27_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq27_2'].isnull()) & (test_2017['aq27_1'] == 3), 'aq27_2'] = temp_2017['aq27_2'].astype('int').quantile(q = 0.5)

temp_2017 = train_2017[(train_2017['aq27_2'].notnull()) & (train_2017['aq27_1'] == 4)]
train_2017.loc[(train_2017['aq27_2'].isnull()) & (train_2017['aq27_1'] == 4), 'aq27_2'] = temp_2017['aq27_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq27_2'].isnull()) & (test_2017['aq27_1'] == 4), 'aq27_2'] = temp_2017['aq27_2'].astype('int').quantile(q = 0.5)

test_2017.loc[(test_2017['aq32_1'] == 33), 'aq32_1'] = 3

temp_2017 = train_2017[(train_2017['aq32_2'].notnull()) & (train_2017['aq32_1'] == 3)]
train_2017.loc[(train_2017['aq32_2'].isnull()) & (train_2017['aq32_1'] == 3), 'aq32_2'] = temp_2017['aq32_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq32_2'].isnull()) & (test_2017['aq32_1'] == 3), 'aq32_2'] = temp_2017['aq32_2'].astype('int').quantile(q = 0.5)

temp_2017 = train_2017[(train_2017['aq41_2'].notnull()) & (train_2017['aq41_1'] == 3)]
train_2017.loc[(train_2017['aq41_2'].isnull()) & (train_2017['aq41_1'] == 3), 'aq41_2'] = temp_2017['aq41_2'].astype('int').quantile(q = 0.5)
test_2017.loc[(test_2017['aq41_2'].isnull()) & (test_2017['aq41_1'] == 3), 'aq41_2'] = temp_2017['aq41_2'].astype('int').quantile(q = 0.5)

In [4]:
# bq 문항 결측치 처리
train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1a'].isnull()), 'bq4_1a'] = '없음'

train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2017.loc[(train_2017['bq4'] == 2) & (train_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2017.loc[(test_2017['bq4'] == 2) & (test_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

for index in train_2017.loc[(train_2017['bq4_1b'].isnull())]['bq4_1a'].index:
    value = train_2017.iloc[index]['bq4_1a']

    if len(train_2017[(train_2017['bq4_1a'] == value) & (train_2017['bq4_1b'].notnull()) & (train_2017['bq4_1b'] != '없음') & (train_2017['bq4_1b'] != '없다')]) > 0:
         train_2017.loc[index, 'bq4_1b'] = train_2017[(train_2017['bq4_1a'] == value) & (train_2017['bq4_1b'].notnull()) & (train_2017['bq4_1b'] != '없음') & (train_2017['bq4_1b'] != '없다')]['bq4_1b'].value_counts().index[0]
            
for index in test_2017.loc[(test_2017['bq4_1b'].isnull())].index:
    value = test_2017.iloc[index]['bq4_1a']
    
    if len(test_2017[(test_2017['bq4_1a'] == value) & (test_2017['bq4_1b'].notnull()) & (test_2017['bq4_1b'] != '없음') & (test_2017['bq4_1b'] != '없다')]) > 0:
        test_2017.loc[index, 'bq4_1b'] = test_2017[(test_2017['bq4_1a'] == value) & (test_2017['bq4_1b'].notnull()) & (test_2017['bq4_1b'] != '없음') & (test_2017['bq4_1b'] != '없다')]['bq4_1b'].value_counts().index[0]             

train_2017.loc[(train_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2017.loc[(test_2017['bq4_1b'].isnull()), 'bq4_1b'] = '없음'

train_2017.loc[(train_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2017.loc[(test_2017['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

for col in test_2017:
    train_2017[col].replace('없다','없음', inplace = True)
    test_2017[col].replace('없다','없음', inplace = True)    

train_2017.loc[(train_2017['bq5'] == 2) & (train_2017['bq5_1'].isnull()),'bq5_1'] = 0
test_2017.loc[(test_2017['bq5'] == 2) & (test_2017['bq5_1'].isnull()),'bq5_1'] = 0

train_2017.loc[(train_2017['bq5'] == 2) & (train_2017['bq5_2'].isnull()),'bq5_2'] = '없음'
test_2017.loc[(test_2017['bq5'] == 2) & (test_2017['bq5_2'].isnull()),'bq5_2'] = '없음'

train_2017.loc[train_2017['bq5_2'].isnull(), 'bq5_2'] = '없음'
test_2017.loc[test_2017['bq5_2'].isnull(), 'bq5_2'] = '없음'

train_2017['bq12_1'] = train_2017['bq12_1'].fillna(0)
test_2017['bq12_1'] = test_2017['bq12_1'].fillna(0)

train_2017['bq12_2'] = train_2017['bq12_2'].fillna(0)
test_2017['bq12_2'] = test_2017['bq12_2'].fillna(0)

train_2017['bq12_3'] = train_2017['bq12_3'].fillna(0)
test_2017['bq12_3'] = test_2017['bq12_3'].fillna(0)

train_2017['bq12_4'] = train_2017['bq12_4'].fillna(0)
test_2017['bq12_4'] = test_2017['bq12_4'].fillna(0)

train_2017['bq12_5'] = train_2017['bq12_5'].fillna(0)
test_2017['bq12_5'] = test_2017['bq12_5'].fillna(0)

train_2017.loc[train_2017['bq19_1'].isnull(), 'bq19_1'] = '없음'
train_2017.loc[train_2017['bq19_1'].isnull(), 'bq19_1'] = '없음'

test_2017.loc[test_2017['bq30'].isnull(), 'bq30'] = '없음'

for index in train_2017[(train_2017['bq31'].isnull())].index:
    # 산업 유형
    industry_type = train_2017.iloc[index]['bq1']
    # 유사직업명
    a_similar_job = train_2017.iloc[index]['bq30']
    # 직전직업
    before_job = train_2017.iloc[index]['bq32']
    # 전직 가능직업
    can_change_job = train_2017.iloc[index]['bq33']
    

    if len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업 + 전직 가능 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
        #  유사 직업명 + 직전 직업 + 전직 가능 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job)]):
        # 산업 유형 + 유사 직업명
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)]):
        # 산업 유형 + 직전 직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())&(train_2017['bq1'] == industry_type) & (train_2017['bq30'] == a_similar_job) & (train_2017['bq32'] == before_job)].value_counts().index[0]
    elif len(train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)]):
        # 직전 직업 + 전직 가능직업
        train_2017.loc[index, 'bq31'] = train_2017[(train_2017['bq31'].notnull())& (train_2017['bq32'] == before_job) & (train_2017['bq33'] == can_change_job)].value_counts().index[0]

    else:
        train_2017.loc[index, 'bq31'] = '없음'

for index in test_2017[(test_2017['bq31'].isnull())].index:
    # 산업 유형
    industry_type = test_2017.iloc[index]['bq1']
    # 유사직업명
    a_similar_job = test_2017.iloc[index]['bq30']
    # 직전직업
    before_job = test_2017.iloc[index]['bq32']
    # 전직 가능직업
    can_change_job = test_2017.iloc[index]['bq33']

    if len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job) & (
                              test_2017['bq33'] == can_change_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업 + 전직 가능 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job) & (
                        test_2017['bq33'] == can_change_job)]['bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)]):
        # 산업 유형 + 유사 직업명 + 직전 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)][
            'bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job) & (
            test_2017['bq32'] == before_job) & (test_2017['bq33'] == can_change_job)]):
        #  유사 직업명 + 직전 직업 + 전직 가능 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq30'] == a_similar_job) & (
                        test_2017['bq32'] == before_job) & (test_2017['bq33'] == can_change_job)][
            'bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job)]):
        # 산업 유형 + 유사 직업명
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job)]['bq31'].value_counts().index[0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
            test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)]):
        # 산업 유형 + 직전 직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq1'] == industry_type) & (
                        test_2017['bq30'] == a_similar_job) & (test_2017['bq32'] == before_job)].value_counts().index[
            0]
    elif len(test_2017[(test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job) & (
            test_2017['bq33'] == can_change_job)]):
        # 직전 직업 + 전직 가능직업
        test_2017.loc[index, 'bq31'] = test_2017[
            (test_2017['bq31'].notnull()) & (test_2017['bq32'] == before_job) & (
                        test_2017['bq33'] == can_change_job)].value_counts().index[0]

    else:
        test_2017.loc[index, 'bq31'] = '없음'


train_2017.loc[train_2017['bq32'].isnull(), 'bq32'] = '없음' 
test_2017.loc[test_2017['bq32'].isnull(), 'bq32'] = '없음' 

train_2017.loc[(train_2017['bq34'].isnull()), 'bq34'] = '없음'
test_2017.loc[(test_2017['bq34'].isnull()), 'bq34'] = '없음'

train_2017.loc[train_2017['bq38_1'].isnull(), 'bq38_1'] = '없음'
test_2017.loc[test_2017['bq38_1'].isnull(), 'bq38_1'] = '없음'

train_2017.loc[(train_2017['bq39_1'] == 2) & (train_2017['bq40'].isnull()), 'bq40'] = '3'
test_2017.loc[(test_2017['bq39_1'] == 2) & (test_2017['bq40'].isnull()), 'bq40'] = '3'

train_2017['bq41_1'] = train_2017['bq41_1'].fillna(0)
test_2017['bq41_1'] = test_2017['bq41_1'].fillna(0)

train_2017['bq41_2'] = train_2017['bq41_2'].fillna(0)
test_2017['bq41_2'] = test_2017['bq41_2'].fillna(0)

train_2017['bq41_3'] = train_2017['bq41_3'].fillna(0)
test_2017['bq41_3'] = test_2017['bq41_3'].fillna(0)

train_2017['bq33'] = train_2017['bq33'].fillna('없음')
test_2017['bq33'] = test_2017['bq33'].fillna('없음')

### 2017년 데이터 object 컬럼 처리 

In [5]:
for col in train_2017.select_dtypes('object'):
    if train_2017[col].nunique() < 20 :
        train_2017[col] = train_2017[col].astype(float)

for col in ['bq41_1', 'bq41_2', 'bq41_3'] :
    train_2017[col] = train_2017[col].astype(float)

In [6]:
for col in test_2017.select_dtypes('object'):
    if test_2017[col].nunique() < 20 :
        test_2017[col] = test_2017[col].astype(float)

for col in ['idx', 'bq1', 'bq23', 'bq37', 'bq41_1', 'bq41_2', 'bq41_3'] :
    test_2017[col] = test_2017[col].astype(float)

In [7]:
train_2017.to_csv('Data/train/1.preprocessing_train_2017.csv', index = False)
test_2017.to_csv('Data/test/1.preprocessing_test_2017.csv', index = False)